# Calculate Interpolation Objects in Batches with Job Arrays

In this notebook we will walk through how to create interpolation objects that 
are constructed from batches of effective parameter data. This data was generated
by batching voltage control vectors and running each batch in parallel on the HPC cluster.

**NOTE:** Utilizing job arrays allows for an order of magnitude speed up when 
calcualting effective parameters.

Example: Serial job took 168 hours = 7 days -> Parallel job with 10 CPUs/workers took 16.8 hours!

First change base working directory and import relevant modules.

In [1]:
import os
from pathlib import Path
path = Path(os.getcwd())

# update base working directory to QuDiPy
if path.stem != 'QuDiPy':
    base_dir = path.parents[1]
    os.chdir(base_dir)
else:
    base_dir = path
    
import sys
import numpy as np

from qudipy.system import DotArray

## Mimic HPC Batch Job Execution

We can test how the parallel computation would take place on the cluster with multiple CPUs, but 
instead perform a local serial calculation using only one CPU. The bash script that is shown later will
define how a python file is called in a job array compared to this toy example.

### Toy 4 Batch Example

We can mimic a HPC job array by calling a python script in a for loop as follows.
The python script requires two inputs:
1) The current batch to evaluate
2) The total number of batches that are to be run (in parallel on the HPC cluster)

In [2]:
batches = 4

In [3]:
for idx in range(batches):
    batch = idx + 1 # too mimic hpc
    %run parallel_batch_example_2QD.py {batch} {batches}


------------------ Calculating batch 1/4 now ------------------

g_factor evaluation: control vector=[0.2 0.1 0.4]:  92%|█████████▏| 11/12 [00:15<00:01,  1.43s/it]                       

g_factor evaluation: control vector=[0.2 0.1 0.4]: 100%|██████████| 12/12 [00:16<00:00,  1.34s/it]
Exchange HL evaluation: control vector=[0.2 0.1 0.4]:  75%|███████▌  | 9/12 [00:00<00:00, 25.28it/s]                        
Exchange HL evaluation: control vector=[0.2 0.1 0.4]: 100%|██████████| 12/12 [00:00<00:00, 26.09it/s]
Exchange HM evaluation: control vector=[0.2 0.1 0.4]:  75%|███████▌  | 9/12 [00:00<00:00, 24.26it/s]                        
Exchange HM evaluation: control vector=[0.2 0.1 0.4]: 100%|██████████| 12/12 [00:00<00:00, 23.95it/s]

------------------ Calculating batch 2/4 now ------------------

g_factor evaluation: control vector=[0.26666667 0.1        0.4       ]: 100%|██████████| 12/12 [00:17<00:00,  1.47s/it]  
Exchange HL evaluation: control vector=[0.26666667 0.1        0.4     

## Example SLURM Bash Job Submission Script and Instructions

Rather than calling a python driver script to compute batches of effective parameters, 
the HPC can be leveraged to perform the batch calculations in parallel. This requires the following
steps to be taken.

1) Transfer relevant code and/or nextnano/processed data to the desired cluster. 
2) Create or modify bash scripts to be submitted as a job array using SLURM. 

### Moving Files to the Cluster of Interest

1) In not done so already, start by requesting an account to Alliance Canada which 
will grant access to an array of clusters. Start by reading the documentation about 
selecting a desired cluster, best practices, and how to set up Globus file transfering.

2) Copy code using Globus or via git clone/pull commands.
    NOTE: Familiarity with terminal git commands is required.

3) Ensure existing data sets exist in the qudipy_data_library shared data directory, 
otherwise, use Globus to transfer the desired files. Follow Globus instructions 
for file transfers.

### Create a Job Submission Bash Script

An example bash file will be discussed below to help speed up the learning curve 
when working on an HPC. Please read the documentation on Alliance Canada's webpage 
for a detailed description for how to perform typical HPC tasks.

The example bash file used will be: "\Scripts\hpc\job_arrays\eff_param_array_spin_2QD.sh".

NOTE: other template bash scripts for the HPC are located under "\Scripts\hpc".

```#!/bin/sh

#SBATCH --job-name=2QD_spin
#SBATCH --nodes=1   
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=10       # cpu-cores per task (>1 if multi-threaded tasks)
#SBATCH --mem-per-cpu=10G
#SBATCH --time=5:00:00 

### email notification per batch
#SBATCH --mail-user=<email address>
#SBATCH --mail-type=BEGIN,END,FAIL ###,ARRAY_TASKS <- is no longer allowed for email notifications?

### setup job array
#SBATCH --array=1-10
#SBATCH --output=2QD_spin_%A_%a.out

### Setup virtual enviornment with necessary packages
module load python/3.11.5 # check python version on cluster
virtualenv --no-download $SLURM_TMPDIR/qudipy
source $SLURM_TMPDIR/qudipy/bin/activate
pip install --no-index --upgrade pip
pip install --no-index -r QuDiPy/'QuDiPy package list'

### Call the driver script
python QuDiPy/'parallel_spin_calc_2QD.py' $SLURM_ARRAY_TASK_ID $SLURM_ARRAY_TASK_MAX > job_array_2QD_spin_batch_$SLURM_ARRAY_TASK_ID.txt
```

## Construct Effective Parameter Interpolator From Data Set Batches

In [4]:
# define input/output directories
nav_dir = os.path.join('QuDiPy data', 'tutorials')
processed_dir = os.path.join(base_dir, nav_dir,
                        'processed','2QD_processed')
nextnano_dir = os.path.join(base_dir, nav_dir,
                                'nextnano','2QD_dotsep_60nm')

# anticipated number of dots
n_dots = 2      

# define subset of control ranges to perform calcuations
eff_interp_dims = [4,3,4]
ctrl_vals = [np.linspace(0.2, 0.4, eff_interp_dims[0]),
    np.linspace(-0.1, 0.1, eff_interp_dims[1]),
    np.linspace(0.2, 0.4, eff_interp_dims[2])]

# prefix for saved calculated files
file_prefix = 'example_hpc'

dots = DotArray(n_dots, ctrl_ranges=ctrl_vals, calc=False, hpc=[None, batches,'example_hpc_2QD'])
dots.numeric(processed_dir, nextnano_dir, file_prefix)

dots.construct_interpolator()

Note: No effective parameter calcualtions specified. Default = "spin". 

Loading pre-calculated effective parameters: Batch 4 or 4: 100%|██████████| 4/4 [00:00<00:00, 985.27it/s]
Effective parameter interpolator saved as:
	 example_hpc_spin_data_size_[4 3 4]_from_[ 0.2 -0.1  0.2]_to_[0.4 0.1 0.4].pkl
